In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from model import BiagramLanguageModel
import config

In [2]:
text = open('text.txt').read()



chars = list(set(text))
num_chars = len(chars)
vocab_size = num_chars*num_chars
ctoi = lambda c: chars.index(c)
itos = lambda i: chars[i]


encode = lambda l:[num_chars*ctoi(l[i])+ctoi(l[i+1]) for i in range(0,len(l)-1,2)]
decode = lambda l:[itos(i//num_chars)+itos(i%num_chars) for i in l]

data = torch.tensor(encode(text))
n = int(0.9*len(data))
data_train = data[:n]
data_val = data[n:]


def get_batch(split):
    data = data_train if split == "train" else data_val
    idxs = torch.randint(len(data)-config.block_size, (config.batch_size,))
    x = torch.stack([data[i:i+config.block_size] for i in idxs])
    y = torch.stack([data[i+1:i+config.block_size+1] for i in idxs])
    x , y = x.to(config.device) , y.to(config.device)
    return x , y


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\sachi\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\sachi\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\users\sachi\onedrive\documents\github\envs\funchat\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\users\sachi\onedrive\documents\github\envs\funchat\lib\site-packages\traitlets

In [3]:
model = BiagramLanguageModel(vocab_size).to(config.device)

## Training

In [4]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)
eval_interval = 100
max_iters = 5000
eval_iters = 200

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



step 0: train loss 8.7250, val loss 8.7174
step 100: train loss 4.2816, val loss 4.5421
step 200: train loss 3.8516, val loss 4.1962
step 300: train loss 3.6993, val loss 4.0592
step 400: train loss 3.6186, val loss 3.9952
step 500: train loss 3.5249, val loss 3.9313
step 600: train loss 3.4412, val loss 3.8486
step 700: train loss 3.3366, val loss 3.7975
step 800: train loss 3.2568, val loss 3.7278
step 900: train loss 3.1861, val loss 3.6746
step 1000: train loss 3.1242, val loss 3.6191
step 1100: train loss 3.0677, val loss 3.5719
step 1200: train loss 3.0179, val loss 3.5440
step 1300: train loss 2.9661, val loss 3.5360
step 1400: train loss 2.9305, val loss 3.4907
step 1500: train loss 2.8994, val loss 3.4711
step 1600: train loss 2.8729, val loss 3.4585
step 1700: train loss 2.8409, val loss 3.4226
step 1800: train loss 2.8121, val loss 3.4211
step 1900: train loss 2.7792, val loss 3.3922
step 2000: train loss 2.7673, val loss 3.3705
step 2100: train loss 2.7346, val loss 3.3525


In [5]:
PATH = "./models/gpt2.pt"
torch.save(model, PATH)

In [9]:
model = torch.load(PATH)


test_text = 'Hey there! what are you doing now a days? I hope everything is fine. Lets go for a hunt. There was a hurricane in Indonesia few days back. I think, no one got hurt in that but it is still a very bad news'
test_tokens = torch.tensor(encode(test_text))


temp = torch.stack([test_tokens]).to(config.device)

ans = model.generate(temp,10000)

ans = [decode(x.tolist()) for x in ans]

for _ in ans:
    print(''.join(ans[0]))

Hey there! what are you doing now a days? I hope everything is fine. Lets go for a hunt. There was a hurricane in Indonesia few days back. I think, no one got hurt in that but it is still a very bad news
while: look home drawn upper many him
that braves elst, sir; and led hath the queen! Gentleman:
I took her and hundring for hampior
To rest you our his wayballow; then all finded;
he, Edward's daughter fellowly, to purge on amning.

QUEEN MON EON:
He be no man before I causer no death, the danced
To help your here lateful our Rome: of Littles
he were high a brain'd it.

BOF Scrurn terrior I,--

LUCIO:
Her?
Lay of the king.

ISABELLA:
So blows said,
But pating his is my servates. You possessend
the grown by that dolly in their citislipers.

FRIAR LAURENCE:
Bloss EGurse, good, my lieger, I have toly.

CLAUDIO:
The sweet state, peace o'er of my bowly.

LADY PERCY:
The, let her, as infected, and damberland that be train
Yees so out men, and some we and warm.
Well, my none of mine is me.

Q